In [269]:
import pandas as pd
import numpy as np
import sklearn
from scipy.sparse import csr_matrix, save_npz

### Загружаем 2 датасета: один со всеми группами, другой с теми, которые уже посещали пользователи

In [2]:
start_df = pd.read_csv('datasets/start_df.csv')
attend_df = pd.read_csv('datasets/attend.csv')

### Составляем список номеров групп, которые есть в таблице посещаемости, чтобы в дальнейшем убрать их из обучающего набора

In [3]:
in_attend_list = attend_df['уникальный номер группы'].unique()
in_attend_list

array([801346550, 801346551, 801346552, ..., 801373868, 801373869,
       801373870], dtype=int64)

In [4]:
start_df = start_df[start_df.id.isin(in_attend_list) == False]
start_df

,id,level_1,level_2,level_3,level_4,address,district,area,description,online,duration,day_count,evening_count,freevisit_count,morning_count
4050,801355611,Для души,Творчество,Художественно-прикладное творчество,Рукоделие и творчество,"город Москва, Севастопольский проспект, дом 43А",['Юго-Западный'],['Котловка'],Занятия по изучению основ рукоделия и создания...,0,NaN,NaN,NaN,NaN,NaN
4120,801369321,Для ума,Образование,Информационные технологии,Курсы компьютерной грамотности,"город Москва, поселение Рязановское, посёлок З...",['Троицкий и Новомосковский'],['поселениеРязановское'],Занятия по изучению навыков работы на компьюте...,0,NaN,NaN,NaN,NaN,NaN
8231,801367862,Для тела,Физическая активность,Спортивные игры,Волейбол,"город Москва, Чечёрский проезд, дом 22, корпус 1",['Юго-Западный'],['ЮжноеБутово'],"Занятия по игре в волейбол: правила, техника и...",0,NaN,NaN,NaN,NaN,NaN
8916,801358241,Для тела,Физическая активность,Спортивные игры,Волейбол,"город Москва, 4-й Лесной переулок, дом 6",['Центральный'],['Тверской'],"Занятия по игре в волейбол: правила, техника и...",0,NaN,NaN,NaN,NaN,NaN
13577,801360580,Для души,Рисование,Рисование,Различные техники рисования,"город Москва, 5-я улица Ямского Поля, дом 28",['Северный'],['Беговой'],Занятия по изучению основ цветоведения и компо...,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26998,801370473,Для души,Творчество,Красота и стиль,Мастер-класс по уходу за кожей в зрелом возрасте,"город Москва, улица Маршала Катукова, дом 22",['Северо-Западный'],['Строгино'],Занятия по изучению основ ухода за кожей для л...,0,NaN,NaN,NaN,NaN,NaN
26999,801370643,Для ума,Игры,Шахматы и шашки,Шахматы и шашки,"город Москва, улица Маршала Катукова, дом 22",['Северо-Западный'],['Строгино'],Занятия по изучению правил игры в шахматы и ша...,0,NaN,NaN,NaN,NaN,NaN
27000,801370487,Для тела,Физическая активность,ОФП,ОФП,"город Москва, улица Маршала Катукова, дом 22",['Северо-Западный'],['Строгино'],"Занятия по изучению физических упражнений, нап...",0,NaN,NaN,NaN,NaN,NaN
27001,801372232,Для тела,Физическая активность,"Фитнес, тренажеры",Аэробика,"г. Москва, улица Свободы, дом 79",['Северо-Западный'],['СеверноеТушино'],Занятия по изучению физических упражнений под ...,0,NaN,NaN,NaN,NaN,NaN


### Разделяем "Троицкий и Новомосковский" на два разных  АО по району

In [5]:
for i in start_df[start_df['district'].str.contains('Троицкий и Новомосковский') == True].area.unique():
    print(i)

['поселениеРязановское']
['поселениеДесеновское']
['поселениеМосковский']
['поселениеМарушкинское']
['Троицк']
['поселениеКраснопахорское']
['поселениеСосенское']
['поселениеКленовское']
['поселениеРоговское']
['поселениеВнуковское']
['Щербинка']
['поселениеВороновское']
['поселениеНовофедоровское']


##### интернет говорит:
поселение Рязановское - Новомосковский;
поселение Десеновское - Новомосковский;
поселение Московский - Новомосковский;
поселение Марушкинское - Новомосковский;
Троицк - Троицкий;
поселение Краснопахорское - Троицкий;
поселение Сосенское - Новомосковский;
поселение Кленовское - Троицкий;
поселение Роговское - Троицкий;
поселение Внуковское - Новомосковский;
Щербинка - Новомосковский;
поселение Вороновское - Троицкий;
поселение Новофедоровское - Троицкий;

In [6]:
troitsk_list = ["['Троицк']", "['поселениеКраснопахорское']", "['поселениеКленовское']", "['поселениеРоговское']", "['поселениеВороновское']", "['поселениеНовофедоровское']"]
novomsc_list = ["['поселениеРязановское']", "['поселениеДесеновское']", "['поселениеМосковский']", "['поселениеМарушкинское']", "['поселениеСосенское']", "['поселениеВнуковское']", "['Щербинка']"]
start_df.loc[start_df.area.isin(troitsk_list), 'district'] = "['Троицкий']"
start_df.loc[start_df.area.isin(novomsc_list), 'district'] = "['Новомосковский']"
start_df

,id,level_1,level_2,level_3,level_4,address,district,area,description,online,duration,day_count,evening_count,freevisit_count,morning_count
4050,801355611,Для души,Творчество,Художественно-прикладное творчество,Рукоделие и творчество,"город Москва, Севастопольский проспект, дом 43А",['Юго-Западный'],['Котловка'],Занятия по изучению основ рукоделия и создания...,0,NaN,NaN,NaN,NaN,NaN
4120,801369321,Для ума,Образование,Информационные технологии,Курсы компьютерной грамотности,"город Москва, поселение Рязановское, посёлок З...",['Новомосковский'],['поселениеРязановское'],Занятия по изучению навыков работы на компьюте...,0,NaN,NaN,NaN,NaN,NaN
8231,801367862,Для тела,Физическая активность,Спортивные игры,Волейбол,"город Москва, Чечёрский проезд, дом 22, корпус 1",['Юго-Западный'],['ЮжноеБутово'],"Занятия по игре в волейбол: правила, техника и...",0,NaN,NaN,NaN,NaN,NaN
8916,801358241,Для тела,Физическая активность,Спортивные игры,Волейбол,"город Москва, 4-й Лесной переулок, дом 6",['Центральный'],['Тверской'],"Занятия по игре в волейбол: правила, техника и...",0,NaN,NaN,NaN,NaN,NaN
13577,801360580,Для души,Рисование,Рисование,Различные техники рисования,"город Москва, 5-я улица Ямского Поля, дом 28",['Северный'],['Беговой'],Занятия по изучению основ цветоведения и компо...,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26998,801370473,Для души,Творчество,Красота и стиль,Мастер-класс по уходу за кожей в зрелом возрасте,"город Москва, улица Маршала Катукова, дом 22",['Северо-Западный'],['Строгино'],Занятия по изучению основ ухода за кожей для л...,0,NaN,NaN,NaN,NaN,NaN
26999,801370643,Для ума,Игры,Шахматы и шашки,Шахматы и шашки,"город Москва, улица Маршала Катукова, дом 22",['Северо-Западный'],['Строгино'],Занятия по изучению правил игры в шахматы и ша...,0,NaN,NaN,NaN,NaN,NaN
27000,801370487,Для тела,Физическая активность,ОФП,ОФП,"город Москва, улица Маршала Катукова, дом 22",['Северо-Западный'],['Строгино'],"Занятия по изучению физических упражнений, нап...",0,NaN,NaN,NaN,NaN,NaN
27001,801372232,Для тела,Физическая активность,"Фитнес, тренажеры",Аэробика,"г. Москва, улица Свободы, дом 79",['Северо-Западный'],['СеверноеТушино'],Занятия по изучению физических упражнений под ...,0,NaN,NaN,NaN,NaN,NaN


### Приводим датасет к удобному для машины виду

In [7]:
start_df['district'] = start_df['district'].map(lambda x: x.split(',')[0].replace("['", "").replace("'", "").replace("]", "") if type(x) == str else x)
start_df

,id,level_1,level_2,level_3,level_4,address,district,area,description,online,duration,day_count,evening_count,freevisit_count,morning_count
4050,801355611,Для души,Творчество,Художественно-прикладное творчество,Рукоделие и творчество,"город Москва, Севастопольский проспект, дом 43А",Юго-Западный,['Котловка'],Занятия по изучению основ рукоделия и создания...,0,NaN,NaN,NaN,NaN,NaN
4120,801369321,Для ума,Образование,Информационные технологии,Курсы компьютерной грамотности,"город Москва, поселение Рязановское, посёлок З...",Новомосковский,['поселениеРязановское'],Занятия по изучению навыков работы на компьюте...,0,NaN,NaN,NaN,NaN,NaN
8231,801367862,Для тела,Физическая активность,Спортивные игры,Волейбол,"город Москва, Чечёрский проезд, дом 22, корпус 1",Юго-Западный,['ЮжноеБутово'],"Занятия по игре в волейбол: правила, техника и...",0,NaN,NaN,NaN,NaN,NaN
8916,801358241,Для тела,Физическая активность,Спортивные игры,Волейбол,"город Москва, 4-й Лесной переулок, дом 6",Центральный,['Тверской'],"Занятия по игре в волейбол: правила, техника и...",0,NaN,NaN,NaN,NaN,NaN
13577,801360580,Для души,Рисование,Рисование,Различные техники рисования,"город Москва, 5-я улица Ямского Поля, дом 28",Северный,['Беговой'],Занятия по изучению основ цветоведения и компо...,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26998,801370473,Для души,Творчество,Красота и стиль,Мастер-класс по уходу за кожей в зрелом возрасте,"город Москва, улица Маршала Катукова, дом 22",Северо-Западный,['Строгино'],Занятия по изучению основ ухода за кожей для л...,0,NaN,NaN,NaN,NaN,NaN
26999,801370643,Для ума,Игры,Шахматы и шашки,Шахматы и шашки,"город Москва, улица Маршала Катукова, дом 22",Северо-Западный,['Строгино'],Занятия по изучению правил игры в шахматы и ша...,0,NaN,NaN,NaN,NaN,NaN
27000,801370487,Для тела,Физическая активность,ОФП,ОФП,"город Москва, улица Маршала Катукова, дом 22",Северо-Западный,['Строгино'],"Занятия по изучению физических упражнений, нап...",0,NaN,NaN,NaN,NaN,NaN
27001,801372232,Для тела,Физическая активность,"Фитнес, тренажеры",Аэробика,"г. Москва, улица Свободы, дом 79",Северо-Западный,['СеверноеТушино'],Занятия по изучению физических упражнений под ...,0,NaN,NaN,NaN,NaN,NaN


In [8]:
start_df = start_df.drop(['address', 'area', 'description', 'duration', 'day_count', 'evening_count', 'freevisit_count', 'morning_count'], axis=1)
start_df = start_df.drop(start_df[start_df.level_1.isna()].index)
start_df

,id,level_1,level_2,level_3,level_4,district,online
4050,801355611,Для души,Творчество,Художественно-прикладное творчество,Рукоделие и творчество,Юго-Западный,0
4120,801369321,Для ума,Образование,Информационные технологии,Курсы компьютерной грамотности,Новомосковский,0
8231,801367862,Для тела,Физическая активность,Спортивные игры,Волейбол,Юго-Западный,0
8916,801358241,Для тела,Физическая активность,Спортивные игры,Волейбол,Центральный,0
13577,801360580,Для души,Рисование,Рисование,Различные техники рисования,Северный,0
...,...,...,...,...,...,...,...
26998,801370473,Для души,Творчество,Красота и стиль,Мастер-класс по уходу за кожей в зрелом возрасте,Северо-Западный,0
26999,801370643,Для ума,Игры,Шахматы и шашки,Шахматы и шашки,Северо-Западный,0
27000,801370487,Для тела,Физическая активность,ОФП,ОФП,Северо-Западный,0
27001,801372232,Для тела,Физическая активность,"Фитнес, тренажеры",Аэробика,Северо-Западный,0


In [9]:
start_df['level_1'] = start_df['level_1'].map(lambda x: x.lower())
start_df['level_2'] = start_df['level_2'].map(lambda x: x.replace(' (', ' ').replace(')', '').replace(' /', ' ').lower())
start_df['level_3'] = start_df['level_3'].map(lambda x: x.replace(' (', ' ').replace(')', '').replace('/', ' ').replace('.', ' ').replace('-', ' ').replace(',', ' ').replace('ОНЛАЙН ', '').lower())
start_df['level_4'] = start_df['level_4'].map(lambda x: x.replace(' (', ' ').replace(')', '').replace('/', ' ').replace('.', ' ').replace('-', ' ').replace(',', ' ').replace('?', '').replace('ОНЛАЙН ', '').lower())
start_df['district'] = start_df['district'].map(lambda x: x.replace('-', '') if type(x) == str else x)

start_df.head(10)

,id,level_1,level_2,level_3,level_4,district,online
4050,801355611,для души,творчество,художественно прикладное творчество,рукоделие и творчество,ЮгоЗападный,0
4120,801369321,для ума,образование,информационные технологии,курсы компьютерной грамотности,Новомосковский,0
8231,801367862,для тела,физическая активность,спортивные игры,волейбол,ЮгоЗападный,0
8916,801358241,для тела,физическая активность,спортивные игры,волейбол,Центральный,0
13577,801360580,для души,рисование,рисование,различные техники рисования,Северный,0
13763,801370339,для ума,образование,английский язык,английский язык,Северный,0
16088,801369239,для ума,образование,образовательный практикум,гимнастика мозга,СевероВосточный,0
16224,801370093,для души,спецпроект интеллектуальный клуб,интеллектуальный клуб история искусство,история москвы и россии для всех поколений,Центральный,0
16228,801370091,для души,спецпроект интеллектуальный клуб,интеллектуальный клуб творческие мастерские,прикладная живопись,Центральный,0
16235,801370092,для ума,спецпроект интеллектуальный клуб,интеллектуальный клуб информационные технологии,информационные технологии для москвичей,Центральный,0


In [10]:
start_df.loc[start_df.online == 1, 'district'] = 'Онлайн'
start_df = start_df.drop(['online'], axis=1)
start_df.sample(10)

,id,level_1,level_2,level_3,level_4,district
26910,801370413,для ума,образование,психология и коммуникации,психологические тренинги,Западный
25967,801370437,для души,творчество,музыка фольклор,музыка,ЮгоВосточный
24855,801371903,для души,пение,пение,пение,СевероЗападный
16235,801370092,для ума,спецпроект интеллектуальный клуб,интеллектуальный клуб информационные технологии,информационные технологии для москвичей,Центральный
24841,801369786,для тела,физическая активность,гимнастика,здоровая спина,СевероЗападный
26362,801369113,для тела,физическая активность,гимнастика,цигун,СевероВосточный
26745,801368483,для тела,физическая активность,спортивные игры,настольный теннис,Западный
25619,801365830,для тела,физическая активность,скандинавская ходьба,скандинавская ходьба,Западный
26572,801371405,для тела,физическая активность,гимнастика,танцевальная гимнастика,Восточный
25788,801363943,для тела,физическая активность,гимнастика,гимнастика,ЮгоВосточный


In [11]:
new = start_df.drop(columns=['level_1', 'level_2', 'level_3', 'level_4', 'district'])
new['tags'] = start_df['level_1'] + " " + start_df['level_2'] + " " + start_df['level_3'] + " " + start_df['level_4'] + " " + start_df['district']
new = new.reset_index()
new

,index,id,tags
0,4050,801355611,для души творчество художественно прикладное т...
1,4120,801369321,для ума образование информационные технологии ...
2,8231,801367862,для тела физическая активность спортивные игры...
3,8916,801358241,для тела физическая активность спортивные игры...
4,13577,801360580,для души рисование рисование различные техники...
...,...,...,...
2463,26998,801370473,для души творчество красота и стиль мастер кла...
2464,26999,801370643,для ума игры шахматы и шашки шахматы и шашки С...
2465,27000,801370487,для тела физическая активность офп офп СевероЗ...
2466,27001,801372232,для тела физическая активность фитнес тренаже...


### Инициализируем объекта класса  CountVectorizer для перевода слов в числа и обучаем его на новом датасете

In [144]:
from sklearn.feature_extraction.text import CountVectorizer

with open('model_requirements/stopwords-ru.txt') as f:
    stopWords = f.read().splitlines()

cv = CountVectorizer(max_features=50, encoding='koi8r', stop_words=stopWords)

In [145]:
vector = cv.fit_transform(new['tags']).toarray()
vector.shape

C:\Users\79173\Desktop\lct\LCT\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(


(2468, 50)

In [279]:
cv.transform(['привет мать']).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]], dtype=int64)

### Создаем матрицу "похожести"

In [219]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.        , 0.1132277 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.5       ],
       [0.1132277 , 0.        , 1.        , ..., 0.4330127 , 0.5       ,
        0.        ],
       ...,
       [0.        , 0.        , 0.4330127 , ..., 1.        , 0.57735027,
        0.        ],
       [0.        , 0.        , 0.5       , ..., 0.57735027, 1.        ,
        0.        ],
       [0.        , 0.5       , 0.        , ..., 0.        , 0.        ,
        1.        ]])

### Функция для проверки работы модели

In [280]:
def find_similar(item_id: int):
    soul_list, brain_list, body_list = [], [], []
    
    index = new[new['id'] == item_id].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    rec_list = [new.iloc[i[0]].id for i in distances[1:1001]]
    rec_tags = [new['tags'][new[new['id'] == x].index[0]] for x in rec_list]

    for row in rec_tags:
        if row.split()[1] == 'души':
            if len(soul_list) < 10:
                soul_list.append(row)
        elif row.split()[1] == 'ума':
            if len(brain_list) < 10:
                brain_list.append(row)
        else:
            if len(body_list) < 10:
                body_list.append(row)
    
    return soul_list, brain_list, body_list

In [281]:
find_similar(801355611)

[(0, 1.0000000000000002), (2298, 1.0000000000000002), (406, 0.98058067569092), (407, 0.98058067569092), (1557, 0.98058067569092), (1615, 0.98058067569092), (1678, 0.98058067569092), (2297, 0.98058067569092), (224, 0.960768922830523), (232, 0.960768922830523), (417, 0.960768922830523), (430, 0.960768922830523), (770, 0.960768922830523), (1624, 0.960768922830523), (1663, 0.960768922830523), (1821, 0.960768922830523), (738, 0.9434563530497264), (785, 0.9434563530497264), (786, 0.9434563530497264), (1669, 0.9434563530497264), (31, 0.9230769230769232), (342, 0.9230769230769232), (350, 0.9230769230769232), (517, 0.9230769230769232), (563, 0.9230769230769232), (566, 0.9230769230769232), (666, 0.9230769230769232), (667, 0.9230769230769232), (955, 0.9230769230769232), (989, 0.9230769230769232), (1023, 0.9230769230769232), (1026, 0.9230769230769232), (1029, 0.9230769230769232), (1075, 0.9230769230769232), (1081, 0.9230769230769232), (1098, 0.9230769230769232), (1133, 0.9230769230769232), (1134, 

(['для души творчество художественно прикладное творчество рукоделие и творчество ЮгоЗападный',
  'для души творчество художественно прикладное творчество керамика глина  тестопластика ЮгоЗападный',
  'для души творчество художественно прикладное творчество керамика глина  тестопластика ЮгоЗападный',
  'для души творчество художественно прикладное творчество изготовление аксессуаров и декоративных украшений ЮгоЗападный',
  'для души творчество художественно прикладное творчество вязание на спицах ЮгоЗападный',
  'для души творчество художественно прикладное творчество керамика глина  тестопластика ЮгоЗападный',
  'для души творчество художественно прикладное творчество лоскутное шитье ЮгоЗападный',
  'для души творчество художественно прикладное творчество рукоделие и творчество Троицкий',
  'для души творчество художественно прикладное творчество рукоделие и творчество Троицкий',
  'для души творчество художественно прикладное творчество рукоделие и творчество Троицкий'],
 ['для ума и

In [265]:
new[new.id == 801355611].tags.values

array(['для души творчество художественно прикладное творчество рукоделие и творчество ЮгоЗападный'],
      dtype=object)

In [275]:
save_npz("model_requirements/similarity.npz", csr_matrix(similarity))
save_npz("model_requirements/vector.npz", csr_matrix(vector))

In [300]:
import pickle
with open('model_requirements/id_to_group_id.pkl', 'wb') as f:
    pickle.dump(id_to_group_id, f)

In [277]:
with open('model_requirements/count_vectorizer.pkl', 'wb') as f:
    pickle.dump(cv, f)

In [298]:
with open('model_requirements/id_to_kat.pkl', 'wb') as f:
    pickle.dump(dict(new.tags.map(lambda x: x.split()[1])), f)